# Notes
These are just notes I used for constructing the TFP joints

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util
tfp.__version__

In [ ]:
import maxentep

In [ ]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
print(tf.where(bs > 0)[:,1:])


In [ ]:
j = maxentep.recip_norm_mat_dist(np.concatenate((start[np.newaxis,...], start[np.newaxis,...])),
                                 np.concatenate((var[np.newaxis,...], var[np.newaxis,...])))
print(j)

In [ ]:
tfd.JointDistributionSequentialAutoBatched

In [ ]:
print(j.sample(1))
#j.log_prob(j.sample())

In [ ]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]]).astype(np.float32)
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
i = tf.reshape(tf.cast(tf.where(bs > 0)[...,1:], tf.int32), (-1, start.shape[0], 2))
v = tf.gather_nd(bs, i, batch_dims=1)
print(i, v)
b = maxentep.Scatter(i, bs.shape[1:])
bs.shape

In [ ]:
b.forward(v)
print(v, b.inverse(b.forward(v)))

In [ ]:
b.inverse_log_det_jacobian(v, 1)

In [ ]:
tf.map_fn(lambda x: tf.scatter_nd(x[0], x[1], bs.shape[1:]), elems=[i,v],  dtype=tf.float32)

In [ ]:
w = tf.constant(start.astype(np.float32))
w, tf.ones(tf.rank(w))

In [ ]:
i =  tf.keras.Input((1,))
combined = np.stack((start, var))
x  = maxentep.TrainableInputLayer(combined, constraint=maxentep.PositiveMaskedConstraint(combined > 0))(i)

In [ ]:
m = tf.keras.Model(inputs=i, outputs=x)
t = m(tf.constant([1., 1.]))
t

In [ ]:
t[0,0], t[0,1]

In [ ]:
maxentep.recip_norm_mat_dist(t[:1,0], t[:1,1]).sample(5)

In [ ]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_mat_layer(i, start, var, 'test')

In [ ]:
m = tf.keras.Model(inputs=i, outputs=l)

In [ ]:
m(tf.constant([1.]))